In [1]:
import pandas as pd
import numpy as np

Work interference - If you have a mental health condition, do you feel that it interferes with your work?

remote work- Do you work remotely (outside of an office) at least 50% of the time?

benefits - Does your employer provide mental health benefits?

care options - Do you know the options for mental health care your employer provides?

wellness program - Has your employer ever discussed mental health as part of an employee wellness program?

seek help - Does your employer provide resources to learn more about mental health issues and how to seek help?

annonimity - Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment

leave - How easy is it for you to take medical leave for a mental health condition?

mental health conseq - Do you think that discussing a mental health issue with your employer would have negative consequences?

phys_health_consequence - Do you think that discussing a physical health issue with your employer would have negative consequences?


In [3]:
df = pd.read_csv('data.csv')
df.head(2)

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN


In [4]:
df.work_interfere.dropna(inplace=True)

In [5]:
df.self_employed.dropna(inplace=True)

In [6]:
df.drop(['Timestamp', 'comments', 'state'], axis=1, inplace=True)